<a href="https://colab.research.google.com/github/6mini/fancim-project/blob/main/fancim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT를 위한 환경 설정

In [ ]:
#KoBERT를 사용하기 위한 라이브러리 설정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [1]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [3]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

# AI Hub에서 받아온 데이터 전처리하기

- 공포 : 5468 
- 놀람 : 5898 
- 분노 : 5665
- 슬픔 : 5267
- 중립 : 4830
- 행복 : 6037
- 혐오 : 5429

라벨 중, 혐오를 부정으로, 행복을 긍정으로 치환하여 3가지 라벨을 통해서만 모델링을 진행한다.

In [5]:
import pandas as pd

In [6]:
# pd.read_csv로 csv파일 불러오기
df = pd.read_csv('korean_oneoff_chats.csv')
df

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [7]:
#결측치 확인
df.isna().sum()

Sentence    0
Emotion     0
dtype: int64

## Emotion별 특징 분석하기

In [8]:
df['Emotion'].unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

In [9]:
idx = df[df['Emotion'] == '분노'].index
idx2 = df[df['Emotion'] == '공포'].index
idx3 = df[df['Emotion'] == '슬픔'].index
idx4 = df[df['Emotion'] == '놀람'].index
df = df.drop(idx)
df = df.drop(idx2)
df = df.drop(idx3)
df = df.drop(idx4)
df

,Sentence,Emotion
22298,소량은 먹어도 인체에 해롭지 않고요.,중립
22299,그래도 청문회 나온 놈들보다는 정직한 대답 한것 같다,중립
22300,이런일 터지기전에 이미 계약 했을까?,중립
22301,좀더 신중할필요가 있다...,중립
22302,이번엔 베터리 색상은?,중립
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [10]:
df.loc[(df['Emotion']=='혐오'), 'Emotion'] = 0 #혐오를 0으로 표현
df.loc[(df['Emotion']=='중립'), 'Emotion'] = 1 #중립을 1으로 표현
df.loc[(df['Emotion']=='행복'), 'Emotion'] = 2 #행복을 2으로 표현

In [11]:
df.sample(10)

,Sentence,Emotion
25852,북한 비판을 쓰기위해 나라를 북한으로 만드네.,1
35557,가지마라. 저기 말고 좋은데 많다,0
27634,유재석 Jr.의 탄생을 축하드려요~,2
33552,"김고은이 앙칼지고 예민한 홍설 캐릭터를 완전 붕괴시켜놔서, 오연서가 평타만 쳐도 호...",0
29071,혜리가 나온다니 그저 고마울뿐 ㅜㅜ,2
36773,인간의 욕심이 만드는 허상일뿐 귀부인은 완성될 수 없습니다,0
37201,차 살돈도 없으니.. 참 댓글수준 볼만하다ㅋㅋ,0
26935,홍어들 스마트폰 사용자중에 대부분 아이폰 이라고 보면 돼죠,1
37673,A로드가 대단하다 약쟁이인데 그나이까지 꾸역꾸역 사는 것 보니까,0
23097,국회에 앞서 우리 국민부터 각성해야 할 시기입니다,1


In [12]:
#각 행의 데이터를 리스트로 묶어주고 하나의 리스트에 넣기
data_list = []

for q, label in zip(df['Sentence'], df['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [13]:
data_list[0]

[' 소량은 먹어도 인체에 해롭지 않고요.', '1']

# 데이터셋 나누기

In [14]:
from sklearn.model_selection import train_test_split
                                                         
train_li, test_li = train_test_split(data_list, test_size=0.2, random_state=0)

In [15]:
print(len(train_li))
print(len(test_li))

13036
3260


# KoBERT 입력 데이터로 데이터셋 만들기

In [16]:
#BERT 모델에 들어가기 위한 데이터셋을 만들어주는 사용자 클래스 만들기
class BERTdata(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return(len(self.labels))

In [17]:
# KoBERT의 파라미터 세팅
max_len = 64             #텍스트 데이터 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5           #반복학습 수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

# 토큰화

In [18]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
#위의 클래스를 통해 데이터프레임에 있던 데이터를 토큰화해주기
train_data = BERTdata(train_li, 0, 1, tok, max_len, True, False)
test_data = BERTdata(test_li, 0, 1, tok, max_len, True, False)

In [20]:
train_data[0]

(array([   2, 1633, 6280,  517,  364,  523,  517,  364,  517, 7086, 5546,
        7996, 2819, 7828, 2355, 5392, 5436, 6615, 1266, 5400, 5903,  517,
         493,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(24, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [21]:
#torch형식의 데이터셋으로 만들어주어 koBERT에 학습시킬 수 있는 형태로 만들어주기

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# 감정 분류 KoBERT 학습 모델 만들기

In [22]:
class KoBERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##분류할 클래스의 수에 맞춰 조정
                 dr_rate=None,
                 params=None):
        super(KoBERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [23]:
model = KoBERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [24]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [25]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [26]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [27]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [28]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [29]:
train_dataloader

# 모델 학습시키기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

# 새로운 문장들로 테스트하기

위에서 훈련시킨 모델에 새로운 문장을 넣어 감정 분류를 잘 하는지 테스트해볼 수 있다.

In [31]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [56]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTdata(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정")
            elif np.argmax(logits) == 1:
                test_eval.append("중립")
            elif np.argmax(logits) == 2:
                test_eval.append("긍정")

    return test_eval[0]

In [33]:
sentence = input("입력된 문장: ")
predict(sentence)

입력된 문장: 행복하다


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 예측된 감정: 행복


In [115]:
#문장을 입력하면 감정을 예측해주는 입력기
#0 입력시 종료
end = 1
while end == 1 :
    sentence = input("입력된 문장: ")
    if sentence == '0':
        break
    print(predict(sentence))
    print("\n")

입력된 문장: hi


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



중립


입력된 문장: 아 행복해
행복


입력된 문장: 아 슬퍼
행복


입력된 문장: 0


In [65]:
# 셀럽별 변수 생성

import pandas as pd

cjsl = pd.read_csv('cjsl.csv')
dqzb = pd.read_csv('dqzb.csv')
jldn = pd.read_csv('jldn.csv')
jopu = pd.read_csv('jopu.csv')
jseu = pd.read_csv('jseu.csv')
lovm = pd.read_csv('lovm.csv')
osqr = pd.read_csv('osqr.csv')
vaya = pd.read_csv('vaya.csv')
wkdg = pd.read_csv('wkdg.csv')
xmsq = pd.read_csv('xmsq.csv')

In [66]:
# 일괄 전처리 함수

import re

def cleasing(text): 
    repl ='' 
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 자음, 모음 제거 
    text = re.sub(pattern= pattern, repl=repl, string=text) 
    #pattern = '[^\w\s]' # 특수기호 제거 
    pattern = '[^가-히\s]' # 특수기호 제거 
    text = re.sub(pattern= pattern, repl=repl, string=text) 
    pattern = '<[^>]*>' # html 제거 
    text = re.sub(pattern = pattern, repl='',string=text) 
    return text

def preprocessing(df):

    df = df[['회원 코드', '대화 내용']]
    df = df.dropna(axis=0)
    df['대화 내용']= df['대화 내용'].map(lambda x: cleasing(x))
    df['대화 내용'] = df['대화 내용'].apply(lambda x: x.replace('\n',''))
    idx = df[df['대화 내용'] == ' 님이 참여했습니다'].index
    idx2 = df[df['대화 내용'] == ' 님이 나갔습니다'].index
    df = df.drop(idx)
    df = df.drop(idx2)
    return df

cjsl = preprocessing(cjsl)
dqzb = preprocessing(dqzb)
jldn = preprocessing(jldn)
jopu = preprocessing(jopu)
jseu = preprocessing(jseu)
lovm = preprocessing(lovm)
osqr = preprocessing(osqr)
vaya = preprocessing(vaya)
wkdg = preprocessing(wkdg)
xmsq = preprocessing(xmsq)

cjsl, xmsq

(     회원 코드                 대화 내용
 0     CJSL                 안녕하세요
 1     CJSL  얼리어답터분들 두분만이 새로 까셨군요
 3     CJSL          요즘 마리 캣타워입니다
 4     JGXH                      
 6     BYZG               오 저도 깔음
 ...    ...                   ...
 1759  ZMBN              학점보다는 낫죠
 1760  CJSL               이게모람 커엽
 1761  YBJG          귀엽네요 마치 저처럼 
 1762  HCVZ                    어맛
 1763  GILO  시지비에서 파는 팝콘콤보세트요 원인가
 
 [1494 rows x 2 columns],      회원 코드             대화 내용
 0     BYZG    코가쿨라 님이 참여했습니다
 1     BYZG             안녕하세요
 2     XMSQ             안녕하세용
 3     BYZG  이제 장마인데 잠이 안오네요 
 5     LBLA             이몸 등장
 ...    ...               ...
 2320  JPGP       오늘은 거래할수있을까
 2321  ZRAJ            쉐이킷쉐이킷
 2322  WSFL         오 녹차맛 쉐이크
 2323  WSFL     이제 스파르탄큐만 조지면
 2324  XMSQ          베개도 팔아야댐
 
 [2148 rows x 2 columns])

In [67]:
celeb = [cjsl, dqzb, jldn, jopu, jseu, lovm, osqr, vaya, wkdg, xmsq]

for i in celeb:
    print(i.head(1))

  회원 코드  대화 내용
0  CJSL  안녕하세요
  회원 코드           대화 내용
0  BYZG  코가쿨라 님이 참여했습니다
  회원 코드  대화 내용
1  CNJO  이리오너라
  회원 코드  대화 내용
6  HEHJ  아침밥이당
  회원 코드                  대화 내용
1  NTFC  어떻게 나오는지 궁금해서 쳐보는 아무말
  회원 코드 대화 내용
1  COPY      
  회원 코드           대화 내용
0  BYZG  코가쿨라 님이 참여했습니다
  회원 코드 대화 내용
2  KAMF  사랑해요
  회원 코드                                              대화 내용
5  ORNS  알하 방송중이시라 못보시려나요누군가 무슨 얘기를 했는지는 모르지만 답장을 하시면 유...
  회원 코드           대화 내용
0  BYZG  코가쿨라 님이 참여했습니다


In [68]:
# 감정 분석 함수

def gamjung(data):
    return predict(data)

gamjung(cjsl['대화 내용'][0])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


'행복'

In [69]:
# 감정 컬럼 생성

for i in celeb:
    i['감정']= i['대화 내용'].apply(lambda x: gamjung(x))

for i in celeb:
    print(i.head(1))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  회원 코드  대화 내용  감정
0  CJSL  안녕하세요  행복
  회원 코드           대화 내용  감정
0  BYZG  코가쿨라 님이 참여했습니다  행복
  회원 코드  대화 내용  감정
1  CNJO  이리오너라  중립
  회원 코드  대화 내용  감정
6  HEHJ  아침밥이당  행복
  회원 코드                  대화 내용  감정
1  NTFC  어떻게 나오는지 궁금해서 쳐보는 아무말  혐오
  회원 코드 대화 내용  감정
1  COPY        중립
  회원 코드           대화 내용  감정
0  BYZG  코가쿨라 님이 참여했습니다  행복
  회원 코드 대화 내용  감정
2  KAMF  사랑해요  행복
  회원 코드                                              대화 내용  감정
5  ORNS  알하 방송중이시라 못보시려나요누군가 무슨 얘기를 했는지는 모르지만 답장을 하시면 유...  중립
  회원 코드           대화 내용  감정
0  BYZG  코가쿨라 님이 참여했습니다  행복


In [83]:
for i, j in enumerate(celeb):
    print(i)
    print(j[['대화 내용', '감정']].groupby(['감정']).count(), '\n')

0
    대화 내용
감정       
중립    777
행복    509
혐오    208 

1
    대화 내용
감정       
중립    135
행복     80
혐오     24 

2
    대화 내용
감정       
중립    419
행복    269
혐오    126 

3
    대화 내용
감정       
중립     50
행복     51
혐오     15 

4
    대화 내용
감정       
중립     21
행복     14
혐오     10 

5
    대화 내용
감정       
중립     37
행복     19
혐오      8 

6
    대화 내용
감정       
중립     34
행복     28
혐오     10 

7
    대화 내용
감정       
중립    604
행복   2105
혐오    282 

8
    대화 내용
감정       
중립    708
행복    749
혐오    154 

9
    대화 내용
감정       
중립   1124
행복    712
혐오    312 



In [112]:
# 행복도 산출

celeb_name = ['CJSL', 'DQZB', 'JLDN', 'JOPU', 'JSEU', 'LOVM', 'OSQR', 'VAYA', 'WKDG', 'XMSQ']

for i, j in enumerate(celeb):
    j = j[['대화 내용', '감정']].groupby(['감정']).count()
    j = j.reset_index()['대화 내용']
    print('셀럽', celeb_name[i], '의 행복도는', round(j[1] / (j[0] + j[1] + j[2]) * 100, 2), '입니다.')

셀럽 CJSL 의 행복도는 34.07 입니다.
셀럽 DQZB 의 행복도는 33.47 입니다.
셀럽 JLDN 의 행복도는 33.05 입니다.
셀럽 JOPU 의 행복도는 43.97 입니다.
셀럽 JSEU 의 행복도는 31.11 입니다.
셀럽 LOVM 의 행복도는 29.69 입니다.
셀럽 OSQR 의 행복도는 38.89 입니다.
셀럽 VAYA 의 행복도는 70.38 입니다.
셀럽 WKDG 의 행복도는 46.49 입니다.
셀럽 XMSQ 의 행복도는 33.15 입니다.
